We start by importing SimPy as simulator

In [19]:
import simpy
from collections import defaultdict

We define our order list by

In [20]:
#order list [ [sawing time, milling time]]
orders =     [ [4,3], 
               [8,4],
               [3,7],
               [5,6],
               [6,2],
               [3,7],
               [7,2],
               [5,1],
             ]

Please note that sawing always has to come before milling for each order. Some variables to measure the performance:

In [21]:
millDiff = 0 #collect idle times of the mill
millStart = 0 #collect start time of milling
millEnd = 0 #collect end time of milling 

We calculate a simple ordered sequence for the given list:

In [22]:
def calculateSimpleOrderForMachines(orderList):
    execOrder = list()
    for i in range(len(orderList)):
        execOrder.append(i)
    return execOrder

Another option is to sort after the shortest lead time: min(processing_time(saw) + processing_time(mill))

In [23]:
#shortest lead time/processing time over sawing and milling
def calculateShortestLeadTime(orderList):
    lead_times = defaultdict(list)
    #sum up the leadtimes
    for i in range(len(orderList)):
        lead_time = orderList[i][0] + orderList[i][1]
        lead_times[lead_time].append(i)
    #sort the lead_times
    sorted_lead_times = sorted(lead_times.items())
    sorted_order_list = list()
    #get out the values of the leadtime keys
    for key, value in sorted_lead_times:
        for i in range(len(value)):
            sorted_order_list.append(value[i])
    return sorted_order_list

The orders are then executed in this function, saw and mill are resources with limited capacities.

In [24]:
def runOrders(env, orderToDo, saw, mill):
    print('t: {} order {} arrives'.format(env.now, orderToDo))
    with saw.request() as reqSaw:
        yield reqSaw
        #execute order with saw
        print('t: {}: order {} start sawing'.format(env.now, orderToDo))
        yield env.timeout(orderToDo[0])
        print('t: {}: order{} finished sawing'.format(env.now, orderToDo))
    with mill.request() as reqMill:
        yield reqMill
        print('t: {}: order {} start milling'.format(env.now, orderToDo))
        global millDiff
        global millStart
        global millEnd
        millStart = env.now
        millDiff = millDiff + (millStart - millEnd)
        yield env.timeout(orderToDo[1])
        print('t: {}: order{} finished milling'.format(env.now, orderToDo))
        millEnd = env.now

Simulation environment with capacitated resources is initialised:

In [25]:
env = simpy.Environment()
mill = simpy.Resource(env, capacity=1)
saw = simpy.Resource(env, capacity=1)

Execution sequence is calculated for the order list, here, e.g., simply ascending.

In [26]:
#orderedSet = calculateSimpleOrderForMachines(orders) #justd as it arrives
orderedSet = calculateShortestLeadTime(orders) # or using overall shortest lead time

Simulation is initalised with the orders:

In [27]:
millDiff = 0 #collect idle times of the mill
millStart = 0 #collect start time of milling
millEnd = 0 #collect end time of milling 
for i in range(len(orders)):
    env.process(runOrders(env, orders[orderedSet[i]], saw, mill))

Run and print result:

In [28]:
env.run()

print('mill idle time: {}'.format(millDiff))

t: 0 order [5, 1] arrives
t: 0 order [4, 3] arrives
t: 0 order [6, 2] arrives
t: 0 order [7, 2] arrives
t: 0 order [3, 7] arrives
t: 0 order [3, 7] arrives
t: 0 order [5, 6] arrives
t: 0 order [8, 4] arrives
t: 0: order [5, 1] start sawing
t: 5: order[5, 1] finished sawing
t: 5: order [5, 1] start milling
t: 5: order [4, 3] start sawing
t: 6: order[5, 1] finished milling
t: 9: order[4, 3] finished sawing
t: 9: order [4, 3] start milling
t: 9: order [6, 2] start sawing
t: 12: order[4, 3] finished milling
t: 15: order[6, 2] finished sawing
t: 15: order [6, 2] start milling
t: 15: order [7, 2] start sawing
t: 17: order[6, 2] finished milling
t: 22: order[7, 2] finished sawing
t: 22: order [7, 2] start milling
t: 22: order [3, 7] start sawing
t: 24: order[7, 2] finished milling
t: 25: order[3, 7] finished sawing
t: 25: order [3, 7] start milling
t: 25: order [3, 7] start sawing
t: 28: order[3, 7] finished sawing
t: 28: order [5, 6] start sawing
t: 32: order[3, 7] finished milling
t: 32: or